# <a id='toc1_'></a>[Test case of threshold based method](#toc0_)

- Use of NorCP data, ALADIN 12 km
- Daily maxmimum convective precipitation
- Nordic sub-domain
- Checking for extreme days throughout one summer season (2008)


In [ ]:
import sys
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
from dask.distributed import Client
from xtrdet.methods import event_detection_algorithms
from xtrdet.preproc import get_configuration
from xtrdet.utils import open_data
from xtrdet.utils import write_data
from xtrdet.utils import resampling
from xtrdet.utils import spatial_masking
import cartopy.crs as ccrs

In [ ]:
def local_cluster_setup():
    """
    Set up local-pc cluster
    """
    from dask.distributed import LocalCluster
    cluster = LocalCluster(processes=False)
    return cluster

In [ ]:
def slurm_cluster_setup(nodes=1, **kwargs):
    """
    Set up SLURM cluster
    Parameters
    ----------
    nodes: int
        Number of nodes to use
    **kwargs:
        Keyword arguments for cluster specifications
    """
    from dask_jobqueue import SLURMCluster
    cluster = SLURMCluster(**kwargs)
    cluster.scale(nodes)
    return cluster

In [ ]:
def get_args():
    """
    Read configuration file
    Parameters
    ----------
    -
    Returns
    -------
    Input arguments
    """
    import argparse

    # Configuring argument setup and handling
    parser = argparse.ArgumentParser(
        description='Main script for model/obs validation')
    parser.add_argument('--config', '-c',  metavar='name config file',
                        type=str, help='<Required> Full path to config file',
                        required=True)
    return parser.parse_args()

### <a id='toc1_1_1_'></a>[Read configuration file](#toc0_)

In [ ]:
# Get configuration
config_file = '/home/sm_petli/dev/scripts/python/analysis/DEODE/extreme_event_detection/config_main.ini'  # noqa
configuration_dict = get_configuration.get_settings(config_file)
# Create dirs
outdir = configuration_dict['outdir']
if not os.path.exists(configuration_dict['outdir']):
    os.makedirs(outdir)

### <a id='toc1_1_2_'></a>[Starting a cluster](#toc0_)

In [ ]:
# Set up distributed client
if configuration_dict['cluster type'] == 'local':
    cluster = local_cluster_setup()
elif configuration_dict['cluster type'] == 'slurm':
    nnodes = configuration_dict['nodes']
    sl_kwargs = configuration_dict['cluster kwargs']
    cluster = slurm_cluster_setup(nodes=nnodes, **sl_kwargs)
else:
    print("\n\tCluster type not implemented! Exiting..")
    sys.exit()
client = Client(cluster)

In [ ]:
# Settings
var = 'pr'
region = 'Norcp Analysis Domain'

### <a id='toc1_1_3_'></a>[Open, mask & resample target data](#toc0_)

In [ ]:

# Target data
var_conf_trgt = configuration_dict['target data']['variables'][var]
target_conf = open_data.ReadInputData(configuration_dict['target data'])
trgt_data = target_conf.read_data(var)

In [ ]:

masking_trgt_data = spatial_masking.SpatialMasking(trgt_data, var)
trgt_msk_data = masking_trgt_data.get_mask(region, extract_data=True)

In [ ]:
res_freq, res_meth = var_conf_trgt['resample resolution']
resample = resampling.Resampling(res_freq, res_meth)
trgt_mask_resampled = resample.resample(trgt_msk_data)

### <a id='toc1_1_4_'></a>[... and the same for climatology data](#toc0_)

In [ ]:

# Climate data
var_conf_clim = configuration_dict['climatology data']['variables'][var]
clim_conf = open_data.ReadInputData(
    configuration_dict['climatology data'])
clim_data = clim_conf.read_data(var)

In [ ]:
masking_clm_data = spatial_masking.SpatialMasking(clim_data, var)
clim_msk_data = masking_clm_data.get_mask(region, extract_data=True)

In [ ]:
res_freq, res_meth = var_conf_clim['resample resolution']
resample = resampling.Resampling(res_freq, res_meth)
clim_mask_resampled = resample.resample(clim_msk_data)

## <a id='toc1_2_'></a>[Running event detection algorithm](#toc0_)

### <a id='toc1_2_1_'></a>[Work flow](#toc0_)
- Compute all grid point percentile threshold from climatology data
- Sum field values above threshold per day
- Extract top N % of summed daily fields - that's you're detected events!

In [ ]:
method = configuration_dict['detection method']
method_args = configuration_dict['method args']

In [ ]:
event_detection = event_detection_algorithms.ExtremeDetectionAlgorithm(method, method_args, clim_mask_resampled, var, trgt_mask_resampled, var)

In [ ]:
id_events = event_detection.run_algorithm()

In [ ]:
id_events

### Save data

In [ ]:
fname = 'test_output_230304'

In [ ]:
write = write_data.WriteData(id_events, method, outdir, fname)
write.save_data()

## Visualization

In [ ]:
fg = id_events[var].plot.pcolormesh(
    x='lon', y='lat',
    transform=ccrs.PlateCarree(),
    subplot_kws={
        'projection': ccrs.LambertConformal(
            central_longitude=10.0, central_latitude=60.6
        )
    },
    vmin=0.5,
    cmap='YlGnBu',
    col='time',
    col_wrap=3,
    robust=True,
    aspect=.4,
    cbar_kwargs={
        'orientation': 'horizontal',
        'pad': 0.01,
        'aspect': 40,
        'shrink': 0.8,
    },
)
fg.map(lambda: plt.gca().coastlines())

In [ ]:
cluster.close()